In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import os

global no_places, no_photos, no_menus, no_times
no_places = 0
no_photos = 0
no_menus = 0
no_times = 0

def crawl_hospital_info(json):
    global no_places, no_photos, no_menus, no_times
    
    data = json.copy()
    data["imgUrls"] = []

    webDriver = webdriver.Chrome()

    duty_name = json.get('dutyname', '')
    url = f"https://map.naver.com/p/search/{duty_name}"
    webDriver.get(url)
    wait = WebDriverWait(webDriver, 5)

    try:
        iframe_element = wait.until(EC.visibility_of_element_located((By.ID, "entryIframe")))
        webDriver.switch_to.frame(iframe_element)
    except Exception as e:
        no_places += 1
        return data

    try:
        body_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.CB8aP > div")))
        img_elements = body_element.find_elements(By.TAG_NAME, "img")
        data["imgUrls"] = [img.get_attribute("src") for img in img_elements if img.get_attribute("src")]
        for img in img_elements:
            img_url = img.get_attribute("src")
            print("병원 사진:", img_url)
    except Exception as e:
        no_photos += 1
        print("병원 사진 크롤링 실패:", e)

    return data

def process_json_files(input_dir, output_dir):
    global no_places, no_photos, no_menus, no_times

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    file_results = []

    for file_name in os.listdir(input_dir):
        if file_name.endswith('.json'):
            file_path = os.path.join(input_dir, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                hospitals_data = json.load(file)

            for hospital in hospitals_data['DATA']:
                info = crawl_hospital_info(hospital)
                file_results.append(info)

    output_file = os.path.join(output_dir, 'hospitalData_imgUpdate.json')
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump({"DESCRIPTION": hospitals_data["DESCRIPTION"], "DATA": file_results}, file, ensure_ascii=False, indent=4)

input_dir = 'C:/Users/Myhome/Desktop/Crawling'
output_dir = 'C:/Users/Myhome/Desktop/Crawling'
process_json_files(input_dir, output_dir)